Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Damir"
COLLABORATORS = ""

---

# Adding existing constraints

Before beginning, please copy the files out of the Session09 directory and into the `pyrbc_202103_notebooks/inputs` directory. Ignore this step if you ran the `adding_constraints` notebook first and you've already copied these files over.

In [ ]:
# You have to rerun this cell each time you start a new notebook or do a "factory reset".
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.mount_pyrosetta_install() #Instead of pyrosettacolabsetup.setup
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

Drive already mounted at /content/google_drive; to attempt to forcibly remount, call drive.mount("/content/google_drive", force_remount=True).
Notebook is set for PyRosetta use in Colab.  Have fun!


Now cd into the correct directory

In [ ]:
%cd google_drive/MyDrive/temp_pyrbc_202103_notebooks

/content/google_drive/MyDrive/temp_pyrbc_202103_notebooks


We can pass the constraints file as a command line argument.

In [ ]:
from pyrosetta import *
init("-constraints:cst_fa_file inputs/h_bonds.cst")

PyRosetta-4 2021 [Rosetta PyRosetta4.MinSizeRel.python37.ubuntu 2021.21+release.882e5c1ab85c8c251fce4eb3e1e0504af590786a 2021-05-26T14:40:53] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python37.ubuntu r284 2021.21+release.882e5c1ab85 882e5c1ab85c8c251fce4eb3e1e0504af590786a http://www.pyrosetta.org 2021-05-26T14:40:53
core.init: command: PyRosetta -constraints:cst_fa_file inputs/h_bonds.cst -database /content/google_drive/MyDrive/prefix/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=502453676 seed_offset=0 real_seed=502453676
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=502453676 RG_type=mt19937


First, define the score function.

In [ ]:
sfxn = get_fa_scorefxn()

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard res

Let's see the effects of constraints on the scoring of ubiquitin.

In [ ]:
pose = pose_from_pdb("inputs/1ubq.pdb")
sfxn(pose)

core.import_pose.import_pose: File 'inputs/1ubq.pdb' automatically determined to be of type PDB
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /content/google_drive/MyDrive/prefix/lib/python3.7/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/content/google_drive/MyDrive/prefix/lib/python3.7/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.325622 seconds to load from binary


32.67775371933962

Let's look at the individual terms.

In [ ]:
sfxn.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.647    -397.647
 fa_rep                       0.550     103.707      57.039
 fa_sol                       1.000     242.952     242.952
 fa_intra_rep                 0.005     355.469       1.777
 fa_intra_sol_xover4          1.000      16.826      16.826
 lk_ball_wtd                  1.000      -8.756      -8.756
 fa_elec                      1.000    -113.091    -113.091
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -18.828     -18.828
 hbond_lr_bb                  1.000     -23.132     -23.132
 hbond_bb_sc                  1.000      -7.389      -7.389
 hbond_sc                     1.000      -1.549      -1.549
 dslf_fa13                    1.250       0.000       0.000
 omega  

Let's prime the score function with a constraint score term.

In [ ]:
from pyrosetta.rosetta.core.scoring import *
sfxn.set_weight(atom_pair_constraint, 1.0)

We see the new term added to the score function.

In [ ]:
sfxn(pose)
sfxn.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.647    -397.647
 fa_rep                       0.550     103.707      57.039
 fa_sol                       1.000     242.952     242.952
 fa_intra_rep                 0.005     355.469       1.777
 fa_intra_sol_xover4          1.000      16.826      16.826
 lk_ball_wtd                  1.000      -8.756      -8.756
 fa_elec                      1.000    -113.091    -113.091
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -18.828     -18.828
 hbond_lr_bb                  1.000     -23.132     -23.132
 hbond_bb_sc                  1.000      -7.389      -7.389
 hbond_sc                     1.000      -1.549      -1.549
 dslf_fa13                    1.250       0.000       0.000
 atom_pa

But the score of this term is `0.000`. That's because the script doesn't know that it needs to associate the constraint file with this pose. To add the command line constraints to the score function and pose, use:

In [ ]:
from pyrosetta.rosetta.core.scoring.constraints import *
add_fa_constraints_from_cmdline(pose, sfxn)

core.scoring.constraints.util: Constraint choice: inputs/h_bonds.cst
core.scoring.constraints.ConstraintsIO: read constraints from inputs/h_bonds.cst
core.scoring.constraints.ConstraintsIO: Read in 3 constraints


Let's look at the score now. Is the constraint score still `0.000`?
(Note to self: change constraint file and run again.)

In [ ]:
sfxn.show(pose)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -397.647    -397.647
 fa_rep                       0.550     103.707      57.039
 fa_sol                       1.000     242.952     242.952
 fa_intra_rep                 0.005     355.469       1.777
 fa_intra_sol_xover4          1.000      16.826      16.826
 lk_ball_wtd                  1.000      -8.756      -8.756
 fa_elec                      1.000    -113.091    -113.091
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -18.828     -18.828
 hbond_lr_bb                  1.000     -23.132     -23.132
 hbond_bb_sc                  1.000      -7.389      -7.389
 hbond_sc                     1.000      -1.549      -1.549
 dslf_fa13                    1.250       0.000       0.000
 atom_pa

What if we have a mover that completely disrupts the H-bonds?

In [ ]:
pose_perturbed = pose_from_pdb("inputs/1ubq_perturbed.pdb")

core.import_pose.import_pose: File 'inputs/1ubq_perturbed.pdb' automatically determined to be of type PDB


As the new perturbed pose doesn't yet know about the constraints, we need to add them to the pose.

In [ ]:
add_fa_constraints_from_cmdline_to_pose(pose_perturbed)
sfxn.show(pose_perturbed)

core.scoring.constraints.util: Constraint choice: inputs/h_bonds.cst
core.scoring.constraints.ConstraintsIO: read constraints from inputs/h_bonds.cst
core.scoring.constraints.ConstraintsIO: Read in 3 constraints
core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000    -278.300    -278.300
 fa_rep                       0.550     103.026      56.665
 fa_sol                       1.000     192.801     192.801
 fa_intra_rep                 0.005     355.356       1.777
 fa_intra_sol_xover4          1.000      16.827      16.827
 lk_ball_wtd                  1.000     -10.735     -10.735
 fa_elec                      1.000     -60.225     -60.225
 pro_close                    1.250       1.906       2.383
 hbond_sr_bb                  1.000     -16.581     -16.581
 hbond_lr_bb                  1.000 

Does this structure have major score penalties?